```
╔═══════════════════════════════════════════════════════════════════════════════╗
║                                                                               ║
║   ██████╗ ██████╗ ███████╗    ████████╗ ██████╗      ██████╗  ██████╗ ██╗     ║
║   ██╔══██╗██╔═══██╗██╔════╝    ╚══██╔══╝██╔═══██╗    ██╔════╝ ██╔═══██╗██║    ║
║   ██████╔╝██║   ██║█████╗         ██║   ██║   ██║    ███████╗██║   ██║██║     ║
║   ██╔═══╝ ██║   ██║██╔══╝         ██║   ██║   ██║    ╚════██║██║╠══██║██║     ║
║   ██║     ╚██████╔╝██║            ██║   ╚██████╔╝    ███████║╚██████╔╝███████╗║
║   ╚═╝      ╚═════╝ ╚═╝            ╚═╝    ╚═════╝     ╚══════╝ ╚═════╝ ╚══════╝║
║                                                                               ║
║              Importing PDFs to SQL Server with AI                             ║
║         From Structured Output to "Just Run the CLI"                          ║
║                                                                               ║
║                    PSConf.EU 2025 - Chrissy LeMaire                           ║
║                                                                               ║
╚═══════════════════════════════════════════════════════════════════════════════╝
```

In [ ]:
# Setup - Import the module
Import-Module ./aitools.psd1 -Force

# Set default params for raw output in notebooks
$PSDefaultParameterValues["Invoke-AITool:Raw"] = $true

---
## Part 1: The Old Me (Structured Output Era)

> "I used to treat AI like a database function."

Here's what I thought was the "right" way - strict schemas, structured JSON output, validation on every response.

### The Schema I Built

In [ ]:
# Look at this "perfect" JSON schema I created for pet immunization records
Get-Content ./Tests/pdf/immunization.json | ConvertFrom-Json | ConvertTo-Json -Depth 5

### The PDF We're Extracting From

A dog shot record with owner info, pet details, and vaccination history.

In [ ]:
# Open the PDF to see the source data
Start-Process ./Tests/pdf/immunization.pdf

---
## Part 2: The Context Problem

> Long interactive sessions quietly destroy quality.

When I tried to do this in chat mode:
- Early responses were great
- Later responses drifted
- Hallucinations increased
- Context filled with junk from previous attempts

**The solution?** Short-lived, one-shot execution.

---
## Part 3: Enter the CLI Era

> Command-line AI changed everything.

- One-shot execution
- Fresh context every run  
- No conversational baggage
- Easy automation

### The Tools

| Tool | Command | Best For |
|------|---------|----------|
| Claude Code | `claude` | Complex reasoning, code generation |
| GitHub Copilot | `gh copilot` | Free tier available, GitHub integration |
| Gemini CLI | `gemini` | 1000 free requests/day |
| OpenAI Codex | `codex` | Image attachments, GPT-5 |

---
## Part 4: The Actual Workflow

```
1. CLI reads the PDF (as image)
2. Model extracts meaning  
3. Output is shaped once (JSON)
4. Data lands in SQL Server
```

No long chats. No prompt scaffolding. No schema babysitting.

---
## Demo: PDF → JSON with Claude

Convert the PDF to an image, then extract structured data using the JSON schema as context.

In [ ]:
# Set Claude as default
Set-AIToolDefault -Tool Claude

# Convert PDF to image and extract data
$params = @{
    Prompt  = "Extract pet immunization data from this image"
    Context = "./Tests/pdf/immunization.json"
}

$result = Get-ChildItem ./Tests/pdf/immunization.pdf |
    ConvertTo-AITImage |
    Invoke-AITool @params

# Parse and display
$data = $result | ConvertFrom-Json
$data | Format-List

### See the Vaccinations

In [ ]:
# Show vaccination records in a table
$data.vaccinations | Format-Table -AutoSize

---
## Demo: Free Tier with Copilot + GPT-5-mini

> For reliable results at minimal cost

GitHub Copilot CLI offers free tier access.

In [ ]:
# Switch to Copilot with free GPT-5-mini
$params = @{
    Tool    = "Copilot"
    Model   = "gpt-5-mini"
    Prompt  = "Extract data from this image as JSON matching the context schema."
    Context = "./Tests/pdf/immunization.json"
}

$freeResult = Get-ChildItem ./Tests/pdf/immunization.pdf |
    ConvertTo-AITImage |
    Invoke-AITool @params

$freeResult | ConvertFrom-Json | Select-Object pet_name, owner_name, pet_breed

---
## Demo: Gemini (1000 Free/Day)

In [ ]:
# Use Gemini - 1000 free requests per day!
$params = @{
    Tool    = "Gemini"
    Prompt  = "Extract pet immunization data from this image as JSON"
    Context = "./Tests/pdf/immunization.json"
}

Get-ChildItem ./Tests/pdf/immunization.pdf |
    ConvertTo-AITImage |
    Invoke-AITool @params

---
## Part 5: Landing in SQL Server

> Structure at the edge - unstructured inside

Now let's take that JSON and put it in SQL Server using dbatools.

In [ ]:
# First, get fresh data from Claude
$params = @{
    Tool    = "Claude"
    Prompt  = "Extract pet immunization data from this image"
    Context = "./Tests/pdf/immunization.json"
}

$jsonResult = Get-ChildItem ./Tests/pdf/immunization.pdf |
    ConvertTo-AITImage |
    Invoke-AITool @params

$petData = $jsonResult | ConvertFrom-Json
$petData

In [ ]:
# Import dbatools if available
if (Get-Module -ListAvailable dbatools) {
    Import-Module dbatools

    # Connection to local SQL Server
    $sqlInstance = "localhost"
    $database = "PetRecords"

    # Create the pet record
    $petRecord = [PSCustomObject]@{
        PetName   = $petData.pet_name
        OwnerName = $petData.owner_name
        Breed     = $petData.pet_breed
        ExtractedAt = Get-Date
    }

    # Write to SQL Server
    $petRecord | Write-DbaDataTable -SqlInstance $sqlInstance -Database $database -Table Pets -AutoCreateTable

    # Write vaccinations
    $vaccinations = $petData.vaccinations | ForEach-Object {
        [PSCustomObject]@{
            PetName = $petData.pet_name
            VaccineName = $_.vaccine_name
            Date1 = $_.date_administered_1
            Date2 = $_.date_administered_2
            Date3 = $_.date_administered_3
            Veterinarian = $_.veterinarian
        }
    }

    $vaccinations | Write-DbaDataTable -SqlInstance $sqlInstance -Database $database -Table Vaccinations -AutoCreateTable

    Write-Host "Data written to SQL Server!" -ForegroundColor Green
} else {
    Write-Host "dbatools not installed - showing what would be written:" -ForegroundColor Yellow
    Write-Host ""
    Write-Host "PET RECORD:" -ForegroundColor Cyan
    [PSCustomObject]@{
        PetName = $petData.pet_name
        OwnerName = $petData.owner_name
        Breed = $petData.pet_breed
    } | Format-Table

    Write-Host "VACCINATIONS:" -ForegroundColor Cyan
    $petData.vaccinations | Format-Table vaccine_name, date_administered_1, veterinarian -AutoSize
}

---
## Part 6: Query the Results

In [ ]:
if (Get-Module dbatools) {
    # Query what we just inserted
    $query = @"
SELECT p.PetName, p.OwnerName, p.Breed,
       v.VaccineName, v.Date1, v.Veterinarian
FROM Pets p
JOIN Vaccinations v ON p.PetName = v.PetName
ORDER BY v.VaccineName
"@

    Invoke-DbaQuery -SqlInstance $sqlInstance -Database $database -Query $query | Format-Table -AutoSize
}

---
## Batch Processing: Multiple PDFs

The real power is processing many files at once.

In [ ]:
# Example: Process all PDFs in a folder
# Get-ChildItem ./invoices/*.pdf |
#     ConvertTo-AITImage |
#     Invoke-AITool -Tool Claude -Prompt "Extract invoice data" -Context ./schema/invoice.json |
#     ForEach-Object { $_ | ConvertFrom-Json } |
#     Write-DbaDataTable -SqlInstance localhost -Database Invoices -Table InvoiceData -AutoCreateTable

Write-Host "Pattern for batch PDF processing:" -ForegroundColor Cyan
Write-Host @"

Get-ChildItem ./invoices/*.pdf |
    ConvertTo-AITImage |
    Invoke-AITool -Tool Claude -Prompt "Extract data" -Context ./schema.json |
    ForEach-Object { `$_ | ConvertFrom-Json } |
    Write-DbaDataTable -SqlInstance localhost -Database DB -Table Data -AutoCreateTable

"@

---
## The Cost Reality

| Tool | Model | Cost per 1K calls |
|------|-------|-------------------|
| Claude | Haiku 4.5 | ~$0.33 |
| Copilot | GPT-5-mini | **Free** |
| Gemini | Default | **Free** (1000/day) |
| Claude | Sonnet | ~$3.00 |

> **Tip:** Use `ConvertTo-AITImage` + cheap models for high-volume extraction. Save expensive models for complex reasoning.

---
## What Changed For Me

I stopped trying to control AI.

I started:
- Designing workflows
- Managing context  
- Trusting single-shot execution

**Outcome:** Faster, simpler, more reliable systems.

---
## The Takeaway

AI integration isn't about perfect prompts.

It's about:
- Short context
- Clear inputs
- Strong boundaries
- Disposable execution

> **"From 'AI must behave' to 'AI must ship.'"**

---

## Resources

- **aitools module:** https://github.com/potatoqualitee/aitools
- **dbatools:** https://dbatools.io
- **pdf2img:** https://github.com/potatoqualitee/pdf2img

```
Install-Module aitools
Install-Module dbatools
```